In [2]:
import os
os.getcwd()


'/home/yufeil/final-project-yufei/cleaning'

In [4]:
year = 2021

In [5]:
import os
import requests
import zipfile

def download_and_extract_hmda_data(year: int, save_dir: str = "./data"):
    """
    Download and extract CFPB HMDA LAR data for a given year.

    Args:
        year (int): The target year of the dataset to download (e.g., 2023).
        save_dir (str): The directory to save and extract the data into.
    
    Returns:
        str: Path to the extracted .txt file or None if failed.
    """
    os.makedirs(save_dir, exist_ok=True)
    zip_path = os.path.join(save_dir, f"{year}_lar.zip")
    txt_path = os.path.join(save_dir, f"{year}_lar.txt")
    url = f"https://s3.amazonaws.com/cfpb-hmda-public/prod/dynamic-data/{year}/{year}_lar.zip"

    try:
        print(f"Downloading HMDA LAR data for year {year}...")
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(zip_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(save_dir)

        print(f"Extracted to: {save_dir}")
        return txt_path if os.path.exists(txt_path) else None

    except Exception as e:
        print(f"Failed to download/extract: {e}")
        return None


In [6]:
txt_path = download_and_extract_hmda_data(year)

import dask.dataframe as dd
df = dd.read_csv(txt_path, sep='|', dtype=str, assume_missing=True)
df.head()

Extracted to: ./data


,activity_year,lei,derived_msa_md,state_code,county_code,census_tract,conforming_loan_limit,derived_loan_product_type,derived_dwelling_category,derived_ethnicity,...,denial_reason_2,denial_reason_3,denial_reason_4,tract_population,tract_minority_population_percent,ffiec_msa_md_median_family_income,tract_to_msa_income_percentage,tract_owner_occupied_units,tract_one_to_four_family_homes,tract_median_age_of_housing_units
0,2021,549300HW662MN1WU8550,41940,CA,06085,06085503304,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,7059,95.85,148700,53.03,954,1271,39
1,2021,549300HW662MN1WU8550,41940,CA,06085,06085506202,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,7205,56.31,148700,133.15,1556,1910,52
2,2021,549300HW662MN1WU8550,42644,WA,53033,53033032323,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,5666,28.22,115700,154.66,1535,1601,24
3,2021,549300HW662MN1WU8550,36084,CA,06001,06001423901,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,1982,38.45,125600,87.17,353,701,76
4,2021,549300HW662MN1WU8550,31084,CA,06037,06037267200,C,Conventional:First Lien,Single Family (1-4 Units):Site-Built,Not Hispanic or Latino,...,NaN,NaN,NaN,5658,31.92,80000,171.77,1048,827,42


# Dask Cleaning Data

In [2]:
from dask_jobqueue import SLURMCluster

# Compose SLURM script
cluster = SLURMCluster(queue='caslake', cores=10, memory='40GB', 
                       processes=10, walltime='01:00:00', interface='ib0',
                       job_extra=['--account=macs30123']
                      )

# Request resources
cluster.scale(jobs=1)

/software/python-anaconda-2022.05-el8-x86_64/lib/python3.9/site-packages/distributed/node.py:179: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35573 instead
  warnings.warn(


In [3]:
from dask.distributed import Client

client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://172.25.0.66:35573/status,
Dashboard: http://172.25.0.66:35573/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://172.25.0.66:42291,Workers: 0
Dashboard: http://172.25.0.66:35573/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
import dask.dataframe as dd

In [5]:
# 筛选可能用到的变量
columns_needed = [
    # --- 核心分析变量 ---
    'activity_year',
    'action_taken',                 # 是否批准
    'rate_spread',                 # 高风险贷款利差
    'total_loan_costs',            # 总贷款成本
    'loan_amount',                 # 贷款金额
    'income',                      # 收入
    'debt_to_income_ratio',        # 债务收入比
    'applicant_credit_score_type',           # 信用评分类型
    'applicant_age',               # 年龄

    # --- 人口与种族信息 ---
    'derived_race',                # 派生种族
    'derived_ethnicity',          # 派生族裔
    'tract_minority_population_percent',     # tract 少数族裔比例
    'tract_to_msa_income_percentage',        # tract 收入 vs MSA

    # --- 区域信息 ---
    'derived_msa_md',             # 所属 MSA
    'state_code',                 # 州
    'county_code',                # 县
    'census_tract',               # 普查区

    # --- 放贷机构与产品 ---
    'lei',                        # 放贷机构标识
    'loan_type',                  # 贷款类型（Conventional, FHA 等）
    'derived_loan_product_type',  # 派生贷款产品类型（Conventional, FHA, VA）

    # --- 可用于模型扩展的其他变量 ---
    'applicant_sex',              # 性别
    'applicant_race_1',           # 主申人种族
    'applicant_ethnicity_1',      # 主申人族裔
    'co_applicant_race_1',        # 副申人种族
    'co_applicant_ethnicity_1',   # 副申人族裔
    'loan_purpose',               # 贷款用途
    'loan_term',                  # 贷款期限
    'interest_rate',              # 利率
    'property_value',             # 房产估值
    'occupancy_type',             # 占用类型
    'lien_status',                # 抵押状态
]


In [6]:
import dask.dataframe as dd
import os

# 文件夹和年份范围
data_dir = 'data'
years = range(2018, 2024)  # 2018 到 2023

# 初始化列表用于存放各年份子集 DataFrame
dfs = []

# 遍历每个年份文件
for year in years:
    file_path = os.path.join(data_dir, f"{year}_lar.txt")
    print(f"Reading {file_path}...")

    df_year = dd.read_csv(file_path, sep='|', dtype=str, assume_missing=True)
    
    # 确保列存在再筛选，避免 KeyError
    available_cols = [col for col in columns_needed if col in df_year.columns]
    df_subset = df_year[available_cols]
    
    dfs.append(df_subset)

# 合并所有年份的数据
df_all_years = dd.concat(dfs, axis=0, interleave_partitions=True)


Reading data/2018_lar.txt...
Reading data/2019_lar.txt...
Reading data/2020_lar.txt...
Reading data/2021_lar.txt...
Reading data/2022_lar.txt...
Reading data/2023_lar.txt...


In [7]:
import shutil
import os

dir_path = 'data/hmda_filtered_2018_2023.parquet'
if os.path.isdir(dir_path):
    shutil.rmtree(dir_path)
    print("Parquet directory deleted.")
else:
    print("Directory does not exist.")

Parquet directory deleted.


In [8]:
# 保存为 Parquet 文件（建议保存路径也用 data/）
output_path = 'data/hmda_filtered_2018_2023.parquet'
print(f"Saving combined data to {output_path}...")
df_all_years.to_parquet(
    output_path,
    write_index=False,
)

Saving combined data to data/hmda_filtered_2018_2023.parquet...


In [11]:
# Keep only rows where applicant ethnicity is Hispanic/Latino or Not Hispanic/Latino
df_all_years = df_all_years[df_all_years["applicant_ethnicity_1"].isin(['1', '2'])]

# Define borrower race based on ethnicity and race
def define_borrower_race(row):
    if row["applicant_ethnicity_1"] == '1':
        return "Hispanic/Latino"
    elif row["applicant_ethnicity_1"] == '2':
        if row["applicant_race_1"] == '2':
            return "Asian"
        elif row["applicant_race_1"] == '3':
            return "Black"
        elif row["applicant_race_1"] == '5':
            return "White non-Hispanic/non-Latino"
    return None

# Apply the borrower race definition
df_all_years = df_all_years.copy()
df_all_years["borrower_race"] = df_all_years.apply(define_borrower_race, axis=1, meta=("borrower_race", "object"))

# Keep only valid borrower race classifications
df_all_years = df_all_years[df_all_years["borrower_race"].notnull()]

In [12]:
# Convert specified columns to numeric types, coercing errors to NaN
df_all_years["total_loan_costs"] = dd.to_numeric(df_all_years["total_loan_costs"], errors='coerce')
df_all_years["loan_amount"] = dd.to_numeric(df_all_years["loan_amount"], errors='coerce')
df_all_years["rate_spread"] = dd.to_numeric(df_all_years["rate_spread"], errors='coerce')


# Drop rows with missing values in the specified columns
df_all_years = df_all_years.dropna(
    subset=["total_loan_costs", "loan_amount", "rate_spread"]
)

# Compute fee_shares: total_loan_costs / loan_amount
df_all_years["fee_shares"] = df_all_years["total_loan_costs"] / df_all_years["loan_amount"]

# Compute summary statistics by borrower_race
summary_stats = df_all_years.groupby("borrower_race")[[
    "rate_spread", "fee_shares"
]].agg(["mean", "std", "min", "max", "count"]).compute()


In [15]:
# 保存为 Parquet 文件（建议保存路径也用 data/）
output_path = 'data/hmda_filtered_2018_2023.parquet'
print(f"Saving combined data to {output_path}...")
df_all_years.to_parquet(
    output_path,
    write_index=False,
    schema={"rate_spread": "float64", "fee_shares": "float64", "total_loan_costs": "float64", "loan_amount": "float64"}
)

Saving combined data to data/hmda_filtered_2018_2023.parquet...


In [9]:
import dask.dataframe as dd
import pandas as pd

# Load dataset
df_all_years = dd.read_parquet("data/hmda_filtered_2018_2023.parquet")
df_all_years["activity_year"] = df_all_years["activity_year"].astype(str)

years = ['2018', '2019', '2020', '2021', '2022', '2023']
subset_list = []

for year in years:
    # Filter and compute this year's data as Pandas DF first
    year_df = df_all_years[df_all_years["activity_year"] == year].compute()
    subset_list.append(year_df.head(2000))  # Now safe to use head()

# Combine all subsets
combined_df = pd.concat(subset_list, ignore_index=True)

# Convert to Dask and save
sampled_ddf = dd.from_pandas(combined_df, npartitions=1)

output_path = "data/hmda_subset_2018_2023_first2000.parquet"
print(f"Saving sampled data to {output_path}...")

sampled_ddf.to_parquet(
    output_path,
    write_index=False,
    schema={
        "rate_spread": "float64",
        "fee_shares": "float64",
        "total_loan_costs": "float64",
        "loan_amount": "float64"
    }
)

Saving sampled data to data/hmda_subset_2018_2023_first2000.parquet...


In [13]:
df_all_years.head()

,activity_year,action_taken,rate_spread,total_loan_costs,loan_amount,income,debt_to_income_ratio,applicant_credit_score_type,applicant_age,derived_race,...,co_applicant_race_1,co_applicant_ethnicity_1,loan_purpose,loan_term,interest_rate,property_value,occupancy_type,lien_status,borrower_race,fee_shares
0,2018,1,1.305,7822.87,135000,67,46,1,45-54,White,...,8,5,32,240,4.875,195000,1,1,White non-Hispanic/non-Latino,0.057947
1,2018,1,0.057,6859.57,235000,84,50%-60%,9,45-54,White,...,5,2,32,360,4.375,275000,1,1,Hispanic/Latino,0.029190
5,2018,1,-0.110,4612.60,245000,58,47,1,55-64,Black or African American,...,8,5,32,360,4.375,245000,1,1,Black,0.018827
6,2018,1,0.290,4036.72,205000,77,20%-<30%,2,45-54,White,...,8,5,32,360,4.5,315000,1,1,White non-Hispanic/non-Latino,0.019691
11,2018,1,1.427,4553.34,125000,30,46,1,35-44,White,...,5,1,32,360,5.75,155000,1,1,Hispanic/Latino,0.036427


In [18]:
summary_stats

rate_spread                              \
                                     mean       std      min      max   
borrower_race                                                           
Asian                            0.171116  1.005868   -651.0    336.0   
Black                            0.597386  1.534052   -310.0   1882.0   
Hispanic/Latino                  0.587777  2.339316  -3480.0   2440.0   
White non-Hispanic/non-Latino    0.400598  9.389870 -16575.0  44562.0   

                                        fee_shares                             \
                                  count       mean       std  min         max   
borrower_race                                                                   
Asian                           2595385   0.015152  0.021359  0.0   10.080000   
Black                           2928218   0.026856  0.030084  0.0    7.593102   
Hispanic/Latino                 4631023   0.025748  0.033474  0.0   36.346751   
White non-Hispanic/non-Latino  29009245   0.020370  0.047803  0.0  161.343533   

                                         
                                  count  
borrower_race                            
Asian                           2595385  
Black                           2928218  
Hispanic/Latino                 4631023  
White non-Hispanic/non-Latino  29009245

In [12]:
df_all_years.dtypes

activity_year                        object
action_taken                         object
rate_spread                          object
total_loan_costs                     object
loan_amount                          object
income                               object
debt_to_income_ratio                 object
applicant_credit_score_type          object
applicant_age                        object
derived_race                         object
derived_ethnicity                    object
tract_minority_population_percent    object
tract_to_msa_income_percentage       object
derived_msa_md                       object
state_code                           object
county_code                          object
census_tract                         object
lei                                  object
loan_type                            object
derived_loan_product_type            object
applicant_sex                        object
applicant_race_1                     object
applicant_ethnicity_1           

In [10]:
print(sampled_ddf['activity_year'].unique().compute())

0    2018
1    2019
2    2020
3    2021
4    2022
5    2023
Name: activity_year, dtype: object
